# Załadowanie paczek

In [1]:
import pandas as pd
import numpy as np
import json
from awpy.data import PLACE_DIST_MATRIX, NAV
from awpy.analytics.nav import find_closest_area
from tqdm import tqdm

# Read parquet

In [2]:
states = pd.read_parquet("data/states_test.parquet")
states

,filename,mapName,ctBuyType,tBuyType,winningSide,matchID,roundNum,seconds,ctAlivePlayers,ctEqVal,...,tPlayer_4_isAlive,tPlayer_4_lastPlaceName,tPlayer_4_ID,tPlayer_5_spotters,tPlayer_5_activeWeapon,tPlayer_5_mainWeapon,tPlayer_5_secondaryWeapon,tPlayer_5_isAlive,tPlayer_5_lastPlaceName,tPlayer_5_ID
0,BLAST-Premier-Spring-Final-2022-ence-vs-big-bo...,de_mirage,Full Eco,Full Eco,CT,1,1,0.148438,5,3700,...,1,TSpawn,6,[],Knife,Glock-18,Glock-18,1,TSpawn,9
1,BLAST-Premier-Spring-Final-2022-ence-vs-big-bo...,de_mirage,Full Eco,Full Eco,CT,1,1,0.398438,5,3700,...,1,TSpawn,6,[],Knife,Glock-18,Glock-18,1,TSpawn,9
2,BLAST-Premier-Spring-Final-2022-ence-vs-big-bo...,de_mirage,Full Eco,Full Eco,CT,1,1,0.648438,5,3700,...,1,TSpawn,6,[],Knife,Glock-18,Glock-18,1,TSpawn,9
3,BLAST-Premier-Spring-Final-2022-ence-vs-big-bo...,de_mirage,Full Eco,Full Eco,CT,1,1,0.898438,5,3700,...,1,TSpawn,6,[],Knife,Glock-18,Glock-18,1,TSpawn,9
4,BLAST-Premier-Spring-Final-2022-ence-vs-big-bo...,de_mirage,Full Eco,Full Eco,CT,1,1,1.148438,5,3700,...,1,TSpawn,6,[],Knife,Glock-18,Glock-18,1,TSpawn,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620387,IEM-Katowice-2022-vitality-vs-gambit-bo3,de_inferno,Full Buy,Full Buy,T,364,28,7.062500,1,30100,...,0,BombsiteA,241,[],AK-47,AK-47,Glock-18,1,BombsiteA,242
620388,IEM-Katowice-2022-vitality-vs-gambit-bo3,de_inferno,Full Buy,Full Buy,T,364,28,7.312500,1,30100,...,0,BombsiteA,241,[],AK-47,AK-47,Glock-18,1,BombsiteA,242
620389,IEM-Katowice-2022-vitality-vs-gambit-bo3,de_inferno,Full Buy,Full Buy,T,364,28,7.562500,1,30100,...,0,BombsiteA,241,[],AK-47,AK-47,Glock-18,1,BombsiteA,242
620390,IEM-Katowice-2022-vitality-vs-gambit-bo3,de_inferno,Full Buy,Full Buy,T,364,28,7.812500,1,30100,...,0,BombsiteA,241,[],AK-47,AK-47,Glock-18,1,BombsiteA,242


In [3]:
states.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 620392 entries, 0 to 620391
Columns: 138 entries, filename to tPlayer_5_ID
dtypes: float64(34), int64(48), object(56)
memory usage: 653.2+ MB


## PLACE_DIST

In [4]:
PLACE_DIST_MATRIX.keys()

dict_keys(['de_ancient', 'de_dust2', 'de_inferno', 'de_mirage', 'de_nuke', 'de_overpass', 'de_vertigo'])

In [5]:
PLACE_DIST_MATRIX['de_mirage'].keys()

dict_keys(['CTSpawn', 'BombsiteA', 'TRamp', 'TicketBooth', 'TopofMid', '', 'Shop', 'Apartments', 'BombsiteB', 'PalaceAlley', 'TSpawn', 'Catwalk', 'House', 'SnipersNest', 'Jungle', 'Tunnel', 'BackAlley', 'Middle', 'PalaceInterior', 'TunnelStairs', 'Ladder', 'Stairs', 'SideAlley', 'Scaffolding', 'Truck', 'Connector', 'Balcony'])

# Modyfikacje

## Odległości do bombsite'u

In [6]:
def get_dist(isAlive, mapName, lastPlace, position):
    if isAlive:
        if lastPlace in PLACE_DIST_MATRIX[mapName]:
            newPlace = lastPlace
        else:
            areaID = find_closest_area(mapName, position)['areaId']
            newPlace = NAV[mapName][areaID]["areaName"]
        distToA = PLACE_DIST_MATRIX[mapName][newPlace]['BombsiteA']['geodesic']['median_dist']
        distToB = PLACE_DIST_MATRIX[mapName][newPlace]['BombsiteB']['geodesic']['median_dist']
    else:
        distToA = None
        distToB = None
    return distToA, distToB

In [7]:
all_parts = []
for team in tqdm(['ct', 't'], leave=True, position=0):
    for i in tqdm(range(1, 6), leave=True, position=1):
        data = [ 
            get_dist(isAlive, mapName, lastPlace, [x, y, z])
                for isAlive, mapName, lastPlace, x, y, z in 
                    zip(states[f'{team}Player_{i}_isAlive'],
                        states[f'mapName'], 
                        states[f'{team}Player_{i}_lastPlaceName'],
                        states[f'{team}Player_{i}_x'],
                        states[f'{team}Player_{i}_y'],
                        states[f'{team}Player_{i}_z'])
        ]
        part = pd.DataFrame.from_records(data, columns=[f'{team}Player_{i}_distToA', f'{team}Player_{i}_distToB'])
        all_parts.append(part)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:44<00:00, 22.15s/it]


In [8]:
all_parts[0].tail(, all_parts[1], all_parts[2], all_parts[3], all_parts[4]

(        ctPlayer_1_distToA  ctPlayer_1_distToB
 0              1618.282436         4369.254726
 1              1618.282436         4369.254726
 2              1618.282436         4369.254726
 3              1618.282436         4369.254726
 4              1618.282436         4369.254726
 ...                    ...                 ...
 620387                 NaN                 NaN
 620388                 NaN                 NaN
 620389                 NaN                 NaN
 620390                 NaN                 NaN
 620391                 NaN                 NaN
 
 [620392 rows x 2 columns],
         ctPlayer_2_distToA  ctPlayer_2_distToB
 0              1618.282436         4369.254726
 1              1618.282436         4369.254726
 2              1618.282436         4369.254726
 3              1618.282436         4369.254726
 4              1618.282436         4369.254726
 ...                    ...                 ...
 620387          682.815793         5835.925661
 620388   

## Widziani gracze

In [8]:
def get_spotted_players(*args):
    spotted = set()
    for arg in args:
        spotted.update(json.loads(arg))
    return len(spotted)

In [9]:
spotters_parts = []
for team in tqdm(['ct', 't'], leave=True, position=0):
    data = [ 
        get_spotted_players(spot1, spot2, spot3, spot4, spot5)
                for spot1, spot2, spot3, spot4, spot5 in 
                    zip(states[f'{team}Player_1_spotters'],
                       states[f'{team}Player_2_spotters'],
                       states[f'{team}Player_3_spotters'],
                       states[f'{team}Player_4_spotters'],
                       states[f'{team}Player_5_spotters'])
        ]
    part = pd.DataFrame(data, columns=[f'{team}_spottedPlayers'])
    spotters_parts.append(part)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:19<00:00,  9.60s/it]


In [10]:
states.columns.values

array(['filename', 'mapName', 'ctBuyType', 'tBuyType', 'winningSide',
       'matchID', 'roundNum', 'seconds', 'ctAlivePlayers', 'ctEqVal',
       'tAlivePlayers', 'tEqVal', 'bombsite', 'ct_hp', 't_hp', 'ct_armor',
       't_armor', 'ct_hasHelmet', 't_hasHelmet', 'ct_DecoyGrenade',
       't_DecoyGrenade', 'ct_Flashbang', 't_Flashbang', 'ct_HEGrenade',
       't_HEGrenade', 'ct_SmokeGrenade', 't_SmokeGrenade',
       'ct_fireGrenades', 't_fireGrenades', 'ct_isBlinded', 't_isBlinded',
       'ct_hasDefuse', 't_hasBomb', 'ct_isDefusing', 't_isPlanting',
       'ctPlayer_1_x', 'ctPlayer_1_y', 'ctPlayer_1_z', 'ctPlayer_2_x',
       'ctPlayer_2_y', 'ctPlayer_2_z', 'ctPlayer_3_x', 'ctPlayer_3_y',
       'ctPlayer_3_z', 'ctPlayer_4_x', 'ctPlayer_4_y', 'ctPlayer_4_z',
       'ctPlayer_5_x', 'ctPlayer_5_y', 'ctPlayer_5_z', 'tPlayer_1_x',
       'tPlayer_1_y', 'tPlayer_1_z', 'tPlayer_2_x', 'tPlayer_2_y',
       'tPlayer_2_z', 'tPlayer_3_x', 'tPlayer_3_y', 'tPlayer_3_z',
       'tPlayer_4_x', 'tP

In [11]:
states.drop(columns=states.columns[states.columns.str.endswith(("_x", "_y", "_z", "_isAlive", "_lastPlaceName", "_spotters"))], inplace=True)
states = pd.concat([states, *spotters_parts, *all_parts], axis=1)
states.columns.values

array(['filename', 'mapName', 'ctBuyType', 'tBuyType', 'winningSide',
       'matchID', 'roundNum', 'seconds', 'ctAlivePlayers', 'ctEqVal',
       'tAlivePlayers', 'tEqVal', 'bombsite', 'ct_hp', 't_hp', 'ct_armor',
       't_armor', 'ct_hasHelmet', 't_hasHelmet', 'ct_DecoyGrenade',
       't_DecoyGrenade', 'ct_Flashbang', 't_Flashbang', 'ct_HEGrenade',
       't_HEGrenade', 'ct_SmokeGrenade', 't_SmokeGrenade',
       'ct_fireGrenades', 't_fireGrenades', 'ct_isBlinded', 't_isBlinded',
       'ct_hasDefuse', 't_hasBomb', 'ct_isDefusing', 't_isPlanting',
       'ctPlayer_1_activeWeapon', 'ctPlayer_1_mainWeapon',
       'ctPlayer_1_secondaryWeapon', 'ctPlayer_1_ID',
       'ctPlayer_2_activeWeapon', 'ctPlayer_2_mainWeapon',
       'ctPlayer_2_secondaryWeapon', 'ctPlayer_2_ID',
       'ctPlayer_3_activeWeapon', 'ctPlayer_3_mainWeapon',
       'ctPlayer_3_secondaryWeapon', 'ctPlayer_3_ID',
       'ctPlayer_4_activeWeapon', 'ctPlayer_4_mainWeapon',
       'ctPlayer_4_secondaryWeapon', 'ctPlay

In [12]:
def prepare_features(unprep_df):
    pistols = {'Glock-18', 'USP-S', 'P2000', 'P250', 'Dual Berettas'}
    enhanced_pistols = {'CZ75 Auto', 'Five-SeveN', 'Tec-9', 'R8 Revolver'}
    deagle = 'Desert Eagle'
    shotguns = {'MAG-7', 'XM1014', 'Nova', 'Sawed-Off'}
    machine_guns = {'M249', 'Negev'}
    smgs = {'MP9', 'MP7', 'MP5-SD', 'MAC-10', 'UMP-45', 'PP-Bizon', 'P90'}
    weaker_rifles = {'Galil AR', 'SSG 08', 'FAMAS'}
    lunet_rifles = {'SG 553', 'AUG'}
    sniper_rifle = {'G3SG1', 'SCAR-20', 'AWP'}
    assault_rifle = {'M4A1', 'M4A4', 'AK-47'}
    others = {'Zeus x27', 'Knife', 'C4', 'Molotov', 'Incendiary Grenade', 
              'Smoke Grenade', 'Flashbang', 'Decoy Grenade', 'HE Grenade', ''}
    others.update(shotguns, machine_guns)
    
    def convert_weapons(col):
        for team in ['ct', 't']:
            unprep_df[f"{team}_{col}_Pistol"] = 0
            unprep_df[f"{team}_{col}_EnhancedPistols"] = 0
            unprep_df[f"{team}_{col}_Deagle"] = 0
            unprep_df[f"{team}_{col}_SMG"] = 0
            unprep_df[f"{team}_{col}_WeakAssaultRifle"] = 0
            unprep_df[f"{team}_{col}_LunetRifle"] = 0
            unprep_df[f"{team}_{col}_SniperRifle"] = 0
            unprep_df[f"{team}_{col}_AssaultRifle"] = 0
            unprep_df[f"{team}_{col}_Others"] = 0
            for i in range(1, 6):
                unprep_df[f"{team}_{col}_Pistol"] += unprep_df[f"{team}Player_{i}_{col}"].isin(pistols).astype(int)
                unprep_df[f"{team}_{col}_EnhancedPistols"] = unprep_df[f"{team}Player_{i}_{col}"].isin(enhanced_pistols).astype(int)
                unprep_df[f"{team}_{col}_Deagle"] += (unprep_df[f"{team}Player_{i}_{col}"] == deagle).astype(int)
                unprep_df[f"{team}_{col}_SMG"] += unprep_df[f"{team}Player_{i}_{col}"].isin(smgs).astype(int)
                unprep_df[f"{team}_{col}_WeakAssaultRifle"] += unprep_df[f"{team}Player_{i}_{col}"].isin(weaker_rifles).astype(int)
                unprep_df[f"{team}_{col}_LunetRifle"] += unprep_df[f"{team}Player_{i}_{col}"].isin(lunet_rifles).astype(int)
                unprep_df[f"{team}_{col}_SniperRifle"] += unprep_df[f"{team}Player_{i}_{col}"].isin(sniper_rifle).astype(int)
                unprep_df[f"{team}_{col}_AssaultRifle"] += unprep_df[f"{team}Player_{i}_{col}"].isin(assault_rifle).astype(int)
                unprep_df[f"{team}_{col}_Others"] += unprep_df[f"{team}Player_{i}_{col}"].isin(others).astype(int)
                
    def convert_sec_weapons(col):
        for team in ['ct', 't']:
            unprep_df[f"{team}_{col}_Pistol"] = 0
            unprep_df[f"{team}_{col}_EnhancedPistols"] = 0
            unprep_df[f"{team}_{col}_Deagle"] = 0
            for i in range(1, 6):
                unprep_df[f"{team}_{col}_Pistol"] += unprep_df[f"{team}Player_{i}_{col}"].isin(pistols).astype(int)
                unprep_df[f"{team}_{col}_EnhancedPistols"] = unprep_df[f"{team}Player_{i}_{col}"].isin(enhanced_pistols).astype(int)
                unprep_df[f"{team}_{col}_Deagle"] += (unprep_df[f"{team}Player_{i}_{col}"] == deagle).astype(int)


    convert_weapons('activeWeapon')
    convert_weapons('mainWeapon')
    convert_sec_weapons('secondaryWeapon')
    
    unprep_df['winningSideCT'] = np.where(unprep_df['winningSide'] == 'CT', 1, 0)
    unprep_df['bombsite'] = np.where(unprep_df['bombsite'] == '', 'not_planted', unprep_df['bombsite'])
    unprep_df['ctMinDistToA'] = unprep_df[["ctPlayer_1_distToA", "ctPlayer_2_distToA", "ctPlayer_3_distToA", "ctPlayer_4_distToA", "ctPlayer_5_distToA"]].min(axis=1)
    unprep_df['ctMinDistToB'] = unprep_df[["ctPlayer_1_distToB", "ctPlayer_2_distToB", "ctPlayer_3_distToB", "ctPlayer_4_distToB", "ctPlayer_5_distToB"]].min(axis=1)
    unprep_df['tMinDistToA'] = unprep_df[["tPlayer_1_distToA", "tPlayer_2_distToA", "tPlayer_3_distToA", "tPlayer_4_distToA", "tPlayer_5_distToA"]].min(axis=1)
    unprep_df['tMinDistToB'] = unprep_df[["tPlayer_1_distToB", "tPlayer_2_distToB", "tPlayer_3_distToB", "tPlayer_4_distToB", "tPlayer_5_distToB"]].min(axis=1)
    unprep_df['ctMeanDistToA'] = unprep_df[["ctPlayer_1_distToA", "ctPlayer_2_distToA", "ctPlayer_3_distToA", "ctPlayer_4_distToA", "ctPlayer_5_distToA"]].mean(axis=1)
    unprep_df['ctMeanDistToB'] = unprep_df[["ctPlayer_1_distToB", "ctPlayer_2_distToB", "ctPlayer_3_distToB", "ctPlayer_4_distToB", "ctPlayer_5_distToB"]].mean(axis=1)
    unprep_df['tMeanDistToA'] = unprep_df[["tPlayer_1_distToA", "tPlayer_2_distToA", "tPlayer_3_distToA", "tPlayer_4_distToA", "tPlayer_5_distToA"]].mean(axis=1)
    unprep_df['tMeanDistToB'] = unprep_df[["tPlayer_1_distToB", "tPlayer_2_distToB", "tPlayer_3_distToB", "tPlayer_4_distToB", "tPlayer_5_distToB"]].mean(axis=1)
    unprep_df = pd.concat([unprep_df, pd.get_dummies(unprep_df['mapName']), pd.get_dummies(unprep_df['bombsite'], prefix='bombsite')], axis=1)
    unprep_df['tMinDistToA'] = unprep_df['tMinDistToA'].fillna(9000)
    unprep_df['tMinDistToB'] = unprep_df['tMinDistToB'].fillna(9000)
    unprep_df['tMeanDistToA'] = unprep_df['tMeanDistToA'].fillna(9000)
    unprep_df['tMeanDistToB'] = unprep_df['tMeanDistToB'].fillna(9000)
    return unprep_df

In [13]:
states = prepare_features(states)
states

,filename,mapName,ctBuyType,tBuyType,winningSide,matchID,roundNum,seconds,ctAlivePlayers,ctEqVal,...,de_ancient,de_dust2,de_inferno,de_mirage,de_nuke,de_overpass,de_vertigo,bombsite_A,bombsite_B,bombsite_not_planted
0,BLAST-Premier-Spring-Final-2022-ence-vs-big-bo...,de_mirage,Full Eco,Full Eco,CT,1,1,0.148438,5,3700,...,0,0,0,1,0,0,0,0,0,1
1,BLAST-Premier-Spring-Final-2022-ence-vs-big-bo...,de_mirage,Full Eco,Full Eco,CT,1,1,0.398438,5,3700,...,0,0,0,1,0,0,0,0,0,1
2,BLAST-Premier-Spring-Final-2022-ence-vs-big-bo...,de_mirage,Full Eco,Full Eco,CT,1,1,0.648438,5,3700,...,0,0,0,1,0,0,0,0,0,1
3,BLAST-Premier-Spring-Final-2022-ence-vs-big-bo...,de_mirage,Full Eco,Full Eco,CT,1,1,0.898438,5,3700,...,0,0,0,1,0,0,0,0,0,1
4,BLAST-Premier-Spring-Final-2022-ence-vs-big-bo...,de_mirage,Full Eco,Full Eco,CT,1,1,1.148438,5,3700,...,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
620387,IEM-Katowice-2022-vitality-vs-gambit-bo3,de_inferno,Full Buy,Full Buy,T,364,28,7.062500,1,30100,...,0,0,1,0,0,0,0,1,0,0
620388,IEM-Katowice-2022-vitality-vs-gambit-bo3,de_inferno,Full Buy,Full Buy,T,364,28,7.312500,1,30100,...,0,0,1,0,0,0,0,1,0,0
620389,IEM-Katowice-2022-vitality-vs-gambit-bo3,de_inferno,Full Buy,Full Buy,T,364,28,7.562500,1,30100,...,0,0,1,0,0,0,0,1,0,0
620390,IEM-Katowice-2022-vitality-vs-gambit-bo3,de_inferno,Full Buy,Full Buy,T,364,28,7.812500,1,30100,...,0,0,1,0,0,0,0,1,0,0


In [14]:
states.drop(columns=states.columns[states.columns.str.endswith(("_activeWeapon", "_mainWeapon", "_secondaryWeapon", "_distToA", "_distToB"))],
            inplace=True)
states.columns

Index(['filename', 'mapName', 'ctBuyType', 'tBuyType', 'winningSide',
       'matchID', 'roundNum', 'seconds', 'ctAlivePlayers', 'ctEqVal',
       ...
       'de_ancient', 'de_dust2', 'de_inferno', 'de_mirage', 'de_nuke',
       'de_overpass', 'de_vertigo', 'bombsite_A', 'bombsite_B',
       'bombsite_not_planted'],
      dtype='object', length=108)

In [15]:
states.to_parquet("data/states_prepared_test.parquet")